In [1]:

import os

import pandas as pd
import numpy as np

import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

import pickle

from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

import utils

pd.set_option("display.max_columns", None)
os.chdir("..")

In [2]:
df = pd.read_csv(
    os.path.join(utils.DATA_PATH, utils.DATA["raw"]), sep=";", decimal=",", index_col=0
)

## Processing target variable

In [3]:
df = df[df["value_eur"] != df["value_eur"].max()]
df["log_value_eur"] = np.log1p(df["value_eur"])

fig = px.histogram(df, x="log_value_eur", nbins=50, title="Log Transformed Value EUR")
fig.update_traces(marker_color=utils.COLORS_DICT['blue'], marker_line_color=utils.COLORS_DICT['gray'], marker_line_width=1)

fig.update_layout(
    width=800, 
    height=400,
    plot_bgcolor="white", 
    yaxis=dict(gridcolor='lightgray'),
    xaxis=dict(gridcolor='lightgray'),
)

fig.show()

In [4]:
# Create helper variable is_superstar, based on upper fence of value_eur
q1 = df['value_eur'].quantile(0.25)
q3 = df['value_eur'].quantile(0.75)
iqr = q3 - q1
upper_fence = q3 + 1.5 * iqr

df['is_superstar'] = df['value_eur'] > upper_fence
df['is_superstar'] = df['is_superstar'].astype(int)
df['is_superstar'].value_counts()

is_superstar
0    5133
1     835
Name: count, dtype: int64

In [5]:
fig = px.box(
    df,
    x="is_superstar",
    y="value_eur",
    color="is_superstar",
    title="Value EUR by category",
    color_discrete_sequence=[utils.COLORS_DICT["blue"], utils.COLORS_DICT["yellow"]],
)
fig.update_layout(
    width=500,
    height=400,
    plot_bgcolor="white",
    yaxis=dict(gridcolor="lightgray"),
    legend_title_text="Superstar",
    xaxis_title="Superstar",
    yaxis_title="Value EUR",
)
fig.update_yaxes(tick0=0, dtick=1e7)
fig.show()

## Processing categorical variables

#### Overview

In [6]:
# Show categorical columns
df[df.select_dtypes(include=['object', 'category']).columns].head(3)

,short_name,club_name,league_name,preferred_foot,formation,player_tags
1,R. Araki,Kashima Antlers,Japanese J. League Division 1,Right,mid,NaN
2,Iván Jaime,Futebol Clube de Famalicão,Portuguese Liga ZON SAGRES,Right,mid,NaN
3,Noguera,FC Goa,Indian Super League,Right,mid,NaN


In [7]:
def summarize_categorical(df: pd.DataFrame) -> pd.DataFrame:
    """Summarize categorical columns in a DataFrame
    Calculates unique categories, most and least common category
    and their percentages."""
    summary = {}

    for col in df.select_dtypes(include=["object", "category"]).columns:
        value_counts = df[col].value_counts(dropna=False)
        total_count = len(df[col])

        unique_values = len(value_counts)
        most_common = value_counts.idxmax()
        least_common = value_counts.idxmin()
        most_common_pct = (value_counts.max() / total_count) * 100
        least_common_pct = (value_counts.min() / total_count) * 100

        summary[col] = {
            "Unique Categories": unique_values,
            "Most Common Category": most_common,
            "Most Common Category %": f"{most_common_pct:.2f}%",
            "Least Common Category": least_common,
            "Least Common Category %": f"{least_common_pct:.2f}%",
        }

    return pd.DataFrame(summary).T


summarize_categorical(df)

,Unique Categories,Most Common Category,Most Common Category %,Least Common Category,Least Common Category %
short_name,5785,J. Rodríguez,0.12%,M. Philipp,0.02%
club_name,543,BSC Young Boys,0.18%,Altay SK,0.17%
league_name,46,USA Major League Soccer,4.98%,Hungarian Nemzeti Bajnokság I,0.18%
preferred_foot,2,Right,75.50%,Left,24.50%
formation,4,def,36.65%,GK,9.10%
player_tags,66,NaN,88.12%,"#Dribbler, #Playmaker, #FK Specialist, #Acroba...",0.02%


### Conclussions:
- `short_name` - not included into further analysis because it is assigned to each of the `sofifa_id`
    -  Might be interesting to see additional information, like nationality of the player
- `club_name` - High variability with 543 unique categories
    - Reduction of dimensions might be needed, by e.g. grouping clubs by region
- `league_name` - Similar to club_name, leagues could be grouped by region, tier, or continent
    - Might be excluded due to potential correlation with `club_name`
- `preferred_foot` - Highly skewed towards 'Right', which may affect model performance if not addresses
    - Is going to be converted to binary `preferred_right_foot`
- `formation` - Further analysis is required to determine its influence
    - Potential grouping into defensive, balanced, or attacking categories
- `player_tags` - Highly sparse with 66 unique categories, and the most common tag appears in nearly 88% of entries
    - Is going to be converted into binary `has_tags`, as is extremaly rare, needs further analysis
    - From the business perspective seems to not have big impact on the estimated value

### Variables drop

#### Drop variables with high cardinality|

In [8]:
df.drop(columns=['short_name'], inplace=True)

### Variables encoding

#### To binary variables

In [9]:
# Encode preferred_foot column
df["preferred_right_foot"] = df["preferred_foot"] == "Right"
df["preferred_right_foot"] = df["preferred_right_foot"].astype(int)

# Encode player_tags column into binary column
df["has_tags"] = df["player_tags"].notnull().astype(int)

# Drop transformed columns
df.drop(columns=["player_tags", "preferred_foot"], inplace=True)

In [10]:
# Plot boxplot of log transformed value_eur by preferred right foot
fig = px.box(
    df,
    x="preferred_right_foot",
    y="log_value_eur",
    title="Player value [EUR] transformed by preferred right foot",
)

fig.update_traces(
    marker_color=utils.COLORS_DICT["blue"],
    marker_line_color=utils.COLORS_DICT["gray"],
    marker_line_width=1,
)
fig.update_layout(
    width=800,
    height=400,
    plot_bgcolor="white",
    yaxis=dict(gridcolor="lightgray"),
)

fig.show()

In [11]:
# Conclusion: There is no significant difference in player value based on preferred foot

In [12]:
# Plot boxplot of log transformed value_eur by player tags
fig = px.box(
    df,
    x="has_tags",
    y="log_value_eur",
    title="Player value [EUR] transformed by player tags",
)

fig.update_traces(
    marker_color=utils.COLORS_DICT["blue"],
    marker_line_color=utils.COLORS_DICT["gray"],
    marker_line_width=1,
)
fig.update_layout(
    width=800,
    height=400,
    plot_bgcolor="white",
    yaxis=dict(gridcolor="lightgray"),
    showlegend=False,
)

fig.add_trace(
    go.Scatter(
        x=df["has_tags"],
        y=df["log_value_eur"],
        mode="markers",
        marker=dict(
            color=df["is_superstar"],
            colorscale=[[0, utils.COLORS_DICT["blue"]], [1, utils.COLORS_DICT["yellow"]]],
            size=5,
            showscale=False,
        ),
    )
)


fig.show()

In [13]:
df["has_tags"].value_counts(normalize=True).round(2)

has_tags
0    0.88
1    0.12
Name: proportion, dtype: float64

In [14]:
# Conclusion: Seems like players with tags have higher value
# It is worth to remember that we have only 12% of data with tags, so we need to be careful with this conclusion
# Assumption: Players with tags are more popular and therefore more valuable

### To one-hot encoding

In [15]:
df['formation'].value_counts(normalize=True).round(2)

formation
def    0.37
mid    0.36
att    0.18
GK     0.09
Name: proportion, dtype: float64

In [16]:
# Plot boxplot of log transformed value_eur by formation
fig = px.box(
    df,
    x="formation",
    y="log_value_eur",
    title="Player value [EUR] transformed by formation",
)

mean_values = (
    df.groupby("formation")[["log_value_eur", "value_eur"]]
    .median()
    .round(2)
    .reset_index()
)
for i, row in mean_values.iterrows():
    fig.add_annotation(
        x=row["formation"],
        y=row["log_value_eur"],
        text=f"Median: {row['value_eur'] / 1e6:.1f}M",
        # xanchor='center',
        yanchor="bottom",
    )

fig.update_traces(
    marker_color=utils.COLORS_DICT["blue"],
    marker_line_color=utils.COLORS_DICT["gray"],
    marker_line_width=1,
)

fig.update_layout(
    width=800,
    height=400,
    plot_bgcolor="white",
    yaxis=dict(gridcolor="lightgray"),
)

fig.show()

In [17]:
# Plot 4 histograms of value_eur by formation
fig = px.histogram(
    df,
    x="log_value_eur",
    color="formation",
    nbins=50,
    title="Log Transformed Value EUR by Formation",
)

visa_colors = list(utils.COLORS_DICT.values())

# Change colors of histograms
for i, trace in enumerate(fig.data):
    trace.marker.color = visa_colors[i % len(visa_colors)]

fig.update_layout(
    width=800,
    height=400,
    plot_bgcolor="white",
    yaxis=dict(gridcolor="lightgray"),
    xaxis=dict(gridcolor="lightgray")
)

fig.show()

In [18]:
# Keep formation column for stratified split
df['formation_old'] = df['formation']
df = pd.get_dummies(df, columns=['formation'], drop_first=True)

### To numerical

In [19]:
# Replace categorical, ordinal club_contract_valid_until with contract_remaining_years which is numerical and more informative
current_year = 2022
df['contract_remaining_years'] = df['club_contract_valid_until'] - current_year

df.drop(columns=['club_contract_valid_until'], inplace=True)

df['contract_remaining_years'].value_counts(normalize=True).round(2)

contract_remaining_years
 0    0.34
 1    0.22
 2    0.18
-1    0.15
 3    0.07
 4    0.03
 5    0.00
 6    0.00
 9    0.00
Name: proportion, dtype: float64

In [20]:
df["is_superstar"] = df["is_superstar"].astype(bool)

fig = px.scatter(
    df,
    x="contract_remaining_years",
    y="log_value_eur",
    color="is_superstar",
    title="Player value [EUR] transformed by contract remaining years",
    trendline="ols",
    trendline_scope="overall",
    color_discrete_sequence=[utils.COLORS_DICT["blue"], utils.COLORS_DICT["yellow"]],
)

fig.update_traces(marker=dict(size=5), selector=dict(mode="markers"))
fig.update_traces(line=dict(color="black"), selector=dict(mode="lines"))

fig.update_layout(
    legend_title_text="Is player a superstar",
    legend_title_font_size=11,
)

fig.update_layout(
    width=1000,
    height=500,
    xaxis_title="Contract remaining years",
    yaxis_title="Player value [EUR] - transformed",
    plot_bgcolor="white",
    yaxis=dict(gridcolor="lightgray"),
    xaxis=dict(gridcolor="lightgray"),
)

fig.show()

- Players with contracts about to expire or recently expired have lower market values
- While players with longer-term contracts tend to have higher values
- The higher variance and extreme values for 5-9 years remaining might be influenced by smaller sample sizes
    - It may also highlight how these are less common scenarios, often reserved for highly valued players

### League Name & Club Name Variables

**Challenges:**
- **High cardinality**: Both variables contain many unique values
- **Potential multicollinearity**: Since clubs are nested within leagues, these variables may be correlated
- **Business perspective**: The `club_name` variable is likely more meaningful from a business perspective
- **Correlation limitations**: Pearson correlation cannot be directly calculated for these variables, as they are non-ordinal categorical variables

**Approach:**
1. **Assess correlation using Target Encoding**:
   - Apply Target Encoding to both `league_name` and `club_name` to measure their correlation with the target variable
2. **Select variable with highest influence on target variable**:
   - Choose the variable with the strongest correlation to the target variable for further analysis
3. **Encode using Label Encoding**:
   - Apply Label Encoding to the selected variable
   - This method is simple, effective, and well-suited for high-cardinality categorical variables that are non-ordinal


In [21]:
# Assessing Correlation Using Target Encoding

def calculate_target_encoded_correlation(df: pd.DataFrame, categorical_var: str, target_var: str):
    """
    Calculate the correlation between a target-encoded categorical variable and the target variable.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame
    categorical_var (str): The name of the categorical variable to encode (e.g., 'league_name')
    target_var (str): The name of the target variable (e.g., 'log_value_eur')

    """
    category_target_mean = df.groupby(categorical_var)[target_var].mean()
    categorical_var_encoded = df[categorical_var].map(category_target_mean)
    
    correlation = categorical_var_encoded.corr(df[target_var]).round(2)
    
    print(f"Correlation between {categorical_var} and {target_var}: {correlation}")

calculate_target_encoded_correlation(df, 'league_name', 'log_value_eur') # Lower correlation, variable should be dropped
calculate_target_encoded_correlation(df, 'club_name', 'log_value_eur') # Higher correlation, variable should be selected

Correlation between league_name and log_value_eur: 0.74
Correlation between club_name and log_value_eur: 0.86


In [22]:
label_encoder = LabelEncoder()
df['club_name_encoded'] = label_encoder.fit_transform(df['club_name'])

df.drop(columns=['league_name', 'club_name'], inplace=True)

label_encoder_path = os.path.join(utils.MODELS_PATH, utils.MODELS["label_encoder"])
with open(label_encoder_path, 'wb') as file:
    pickle.dump(label_encoder, file)

### Final processing

Calculate correlations to get rid out of variables that overlaps

In [23]:
corr_matrix = (
    df.select_dtypes(exclude="object").drop(columns=["sofifa_id", "value_eur"]).corr()
)

corr_matrix = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
corr_matrix = corr_matrix.dropna(axis=0, how='all').dropna(axis=1, how='all')

annotation_text = np.where(np.isnan(corr_matrix.values), "", np.round(corr_matrix.values, 2))

fig = ff.create_annotated_heatmap(
    z=corr_matrix.values,
    x=list(corr_matrix.columns),
    y=list(corr_matrix.index),
    annotation_text=annotation_text,
    colorscale=[
        [0, utils.COLORS_DICT['dark_yellow']],    # -1 correlation
        [0.5, 'lightgray'],  # 0 correlation
        [1,  utils.COLORS_DICT['blue']]  # +1 correlation
    ],
    zmin=-1,
    zmax=1,
    showscale=True,
)

fig.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig.show()


In [24]:
# As there are not many columns, we can manually select the columns with high correlation
# More sophisticated methods can be used for larger datasets 
[column for column in corr_matrix.columns if any((corr_matrix[column] > 0.75) & (corr_matrix[column] != 1))]

df.drop(columns=['overall', 'passing', 'shooting', 'height_cm'], inplace=True)

### Missing values

In [25]:
# Look for missing values, display % of missing values
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]
missing_values = missing_values / df.shape[0] * 100

missing_values.round(1).sort_values(ascending=False)

pace         9.1
dribbling    9.1
defending    9.1
physic       9.1
dtype: float64

In [26]:
imputer = SimpleImputer(strategy="mean")
df = df.copy()
df[["pace", "dribbling", "defending", "physic"]] = imputer.fit_transform(
    df[["pace", "dribbling", "defending", "physic"]]
)

In [42]:
# Check changes
corr_matrix = (
    df.select_dtypes(exclude="object").drop(columns=["sofifa_id", "value_eur"]).corr()
)

corr_matrix = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
corr_matrix = corr_matrix.dropna(axis=0, how='all').dropna(axis=1, how='all')

annotation_text = np.where(np.isnan(corr_matrix.values), "", np.round(corr_matrix.values, 2))

fig = ff.create_annotated_heatmap(
    z=corr_matrix.values,
    x=list(corr_matrix.columns),
    y=list(corr_matrix.index),
    annotation_text=annotation_text,
    colorscale=[
        [0, utils.COLORS_DICT['dark_yellow']],    # -1 correlation
        [0.5, 'lightgray'],  # 0 correlation
        [1,  utils.COLORS_DICT['blue']]  # +1 correlation
    ],
    zmin=-1,
    zmax=1,
    showscale=True,
)

fig.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white'
)

fig.show()

In [43]:
# Save processed dataset
df.to_csv(os.path.join(utils.DATA_PATH, utils.DATA["processed"]), sep=";", decimal=",")

In [56]:
target_variable = "log_value_eur"

corr_matrix = (
    df.select_dtypes(exclude="object").drop(columns=["sofifa_id", "is_superstar", "value_eur"]).corr()
)

corr_matrix = corr_matrix[[target_variable]]
corr_matrix.sort_values(by=target_variable, ascending=False, inplace=True)
corr_matrix

# Remove target variable from the list
corr_matrix = corr_matrix.drop(index=target_variable)

fig = px.bar(
    corr_matrix,
    x=target_variable,
    y=corr_matrix.index,
    orientation="h",
    title="Correlation with target variable",
)

fig.update_traces(marker_color=utils.COLORS_DICT["blue"], marker_line_color=utils.COLORS_DICT["gray"], marker_line_width=1)

fig.update_layout(
    width=500,
    height=400,
    plot_bgcolor="white",
    yaxis=dict(gridcolor="lightgray"),
    xaxis=dict(gridcolor="lightgray"),
)

fig.show()

In [62]:
!jupyter nbconvert --to html notebooks\01_data_processing.ipynb --output=01_data_processing.html